# Section 1-5 - Final Checks

We now arrive at the last piece of puzzle - comparing the mean against the median when filling in the training data.

## Pandas - Extracting data

In [58]:
import pandas as pd
import numpy as np

df = pd.read_csv('C:/Vindico/Projects/Data/Kaggle/Competition/Titanic/train.csv')

## Pandas - Cleaning data

In [59]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

As we don't know whether the mean or the median will do better, we calculate both.

In [60]:
age_mean = df['Age'].mean()
age_median = df['Age'].median()

In [61]:
from scipy.stats import mode

mode_embarked = mode(df['Embarked'])[0][0]

In [62]:
print mode_embarked

S


In [63]:
df['Embarked'] = df['Embarked'].fillna(mode_embarked)

df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)

pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]

df = df.fillna(-1)

train_data = df.values

In [64]:
df.head()

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Embarked_C,Embarked_Q,Embarked_S
0,0,1,3,22,1,0,7.2500,1,0,0,1
1,1,2,1,38,1,0,71.2833,0,1,0,0
2,1,3,3,26,0,0,7.9250,0,0,0,1
3,1,4,1,35,1,0,53.1000,0,0,0,1
4,0,5,3,35,0,0,8.0500,1,0,0,1


## Scikit-learn - Training the model

In [65]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

imputer = Imputer(missing_values=-1)

classifier = ExtraTreesClassifier()

pipeline = Pipeline([
    ('imp', imputer),
    ('clf', classifier),
])

We now include the mean-median comparison into our pipeline.

In [66]:
parameter_grid = {
    'imp__strategy': ['mean', 'median'],
    'clf__max_features': [0.5, 1],
    'clf__max_depth': [5, None],
}

In [67]:
grid_search = GridSearchCV(pipeline, parameter_grid, cv=5, verbose=3)
grid_search.fit(train_data[0:,2:], train_data[0:,0])

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean .....
[CV]  clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean, score=0.793296 -   0.0s
[CV] clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean .....
[CV]  clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean, score=0.815642 -   0.0s
[CV] clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean .....
[CV]  clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean, score=0.797753 -   0.0s
[CV] clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean .....
[CV]  clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean, score=0.792135 -   0.0s
[CV] clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean .....
[CV]  clf__max_features=0.5, clf__max_depth=5, imp__strategy=mean, score=0.870056 -   0.0s
[CV] clf__max_features=0.5, clf__max_depth=5, imp__strategy=median ...
[CV]  clf__max_features=0.5, clf__max_depth=5, imp__strateg

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  32 jobs       | elapsed:    0.4s



[CV] clf__max_features=1, clf__max_depth=None, imp__strategy=mean ....
[CV]  clf__max_features=1, clf__max_depth=None, imp__strategy=mean, score=0.820225 -   0.0s
[CV] clf__max_features=1, clf__max_depth=None, imp__strategy=mean ....
[CV]  clf__max_features=1, clf__max_depth=None, imp__strategy=mean, score=0.775281 -   0.0s
[CV] clf__max_features=1, clf__max_depth=None, imp__strategy=mean ....
[CV]  clf__max_features=1, clf__max_depth=None, imp__strategy=mean, score=0.807910 -   0.0s
[CV] clf__max_features=1, clf__max_depth=None, imp__strategy=median ..
[CV]  clf__max_features=1, clf__max_depth=None, imp__strategy=median, score=0.765363 -   0.0s
[CV] clf__max_features=1, clf__max_depth=None, imp__strategy=median ..
[CV]  clf__max_features=1, clf__max_depth=None, imp__strategy=median, score=0.770950 -   0.0s
[CV] clf__max_features=1, clf__max_depth=None, imp__strategy=median ..
[CV]  clf__max_features=1, clf__max_depth=None, imp__strategy=median, score=0.825843 -   0.0s
[CV] clf__max_f

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.4s finished


GridSearchCV(cv=5,
       estimator=Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values=-1, strategy='mean', verbose=0)), ('clf', ExtraTreesClassifier(bootstrap=False, compute_importances=None,
           criterion='gini', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
           min_samples_split=2, n_estimators=10, n_jobs=1, oob_score=False,
           random_state=None, verbose=0))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'clf__max_features': [0.5, 1], 'clf__max_depth': [5, None], 'imp__strategy': ['mean', 'median']},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=3)

In [68]:
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_score_
grid_search.best_params_

{'clf__max_depth': 5, 'clf__max_features': 0.5, 'imp__strategy': 'mean'}

As before, we replace the -1 values in the column Age by the better performer.

In [69]:
df['Age'] = df['Age'].map(lambda x: age_mean if x == -1 else x)

In [79]:
train_data = df.values

In [80]:
model = BaggingClassifier()
model = model.fit(train_data[0:,2:],train_data[0:,0])

## Scikit-learn - Making predictions

In [81]:
df_test = pd.read_csv('C:/Vindico/Projects/Data/Kaggle/Competition/Titanic/test.csv')

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

Similarly we fill in the NAs in the test data with the better performer.

In [82]:
df_test['Age'] = df_test['Age'].fillna(age_mean)

In [83]:
fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)

df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:,1:])

## Pandas - Preparing for submission

In [84]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]


df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('../titanic_Bagging.csv', index=False)